In [1]:
import os

In [2]:
%pwd

'e:\\MLOps Bootcamp\\HuggingFaceSummarizer\\TextSummarizer\\research'

In [3]:
os.chdir("../")
%pwd

'e:\\MLOps Bootcamp\\HuggingFaceSummarizer\\TextSummarizer'

In [4]:
from pathlib import Path
from dataclasses import dataclass 

In [ ]:
@dataclass
class DataTransformationConfig:
  root_dir: Path
  data_path: Path
  tokenizer_name: str    


In [6]:
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_path = Path("config/config.yaml"),
                 params_filepath = Path("params.yaml")):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [ ]:
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

e:\MLOps Bootcamp\HuggingFaceSummarizer\TextSummarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))



In [ ]:
# Terminal veya notebook hücresinde çalıştırın:
#!pip install tiktoken protobuf

In [14]:
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()

data_transformation = DataTransformation(config = data_transformation_config)

data_transformation.convert()


[2025-07-21 19:11:51,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-21 19:11:51,057: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-21 19:11:51,062: INFO: common: created directory at: artifacts]
[2025-07-21 19:11:51,071: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 15117.14 examples/s]
